# TP BERT



## Introduction

Dans ce TP nous allons:
0. Comprendre comment utiliser un *Jupyter Notebook*, un outil très populaire en analyse des données
1. Préparer un jeu de données pour l’apprentissage d’un modèle [BERT](https://arxiv.org/abs/1810.04805)
3. Comprendre comment utiliser un modèle BERT pré-entraîné
4. Créer un modèle de classification en classes multiples qui exploite les représentations cachés d’un encodeur BERT
5. Entraîner ce modèle et tester ses performances


Avec les outils suivants:

1. [PyTorch](https://pytorch.org/docs/stable/index.html) : une bibliothèque Python *open-source* pour l'apprentissage machine. Si vous avez des difficultées avec Pytorch, il est conseillé de jeter un œil aux tutoriels: https://pytorch.org/tutorials/ . Étant donné l'importance de Pytorch, et puisque nous l'utiliserons également dans un autre TP, vous verrez que le temps consacré à lire les tutoriels aura été profitable.
2. [HuggingFace’s Transformers](https://huggingface.co/transformers/) : une bibliothèque basée sur Pytorch pour le traitement automatique des langues et notamment les modèles neuronaux de type Transformer (comme BERT)
3. [HuggingFace’s Tokenizers](https://github.com/huggingface/tokenizers): une bibliothèque basée sur Pytorch pour la tokenisation, explicitement conçue pour travailler avec la bibliothèque Transformers
4. Google Colab, qui héberge ce *Jupyter Notebook*. Avant de commencer le TP, vous pouvez consulter des pages d'introductions [à Colab](https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb#scrollTo=YHI3vyhv5p85) et [aux Notebooks](https://realpython.com/jupyter-notebook-introduction/)


Les machines Colab sont livrées avec un système Linux et un environnement Python avec plusieurs bibliothèques pré-installées comme, par exemple, Pytorch. Néanmoins, si vous êtes sur une machine Colab, il faut d'abord installer les bibliothèques de HuggingFace, qui ne sont pas pré-installées.

Pour exécuter une commande Unix sur un Jupyter Notebook, il faut placer un point d’exclamation avant la commande: `!commande`. 

In [ ]:
!pip install transformers tokenizers

Maintenant nous pouvons importer les bibliothéques principales dont nous aurons besoin. 

In [ ]:
import torch
import transformers

# Managing arrays
import numpy as np

# Plotting tools:
import matplotlib.pyplot as plt
import seaborn as sns
# load the TensorBoard notebook extension
%load_ext tensorboard

**GPU**

L'apprentissage de notre réseau de neurones requiert beaucoup de calculs matriciels. Pour exécuter ces calculs plus rapidement, il est possible d'utiliser un processeur graphique (*GPU*) p. Si vous êtes sur Colab, vous pouvez utiliser un *GPU* en sélectionnant _Runtime -> Change runtime type -> GPU_.

In [ ]:
if torch.cuda.is_available():
  print("GPU is available.")
  device = torch.cuda.current_device()
else:
  print("Will work on CPU.")

### À rendre

Chaque exercice de ce TP demande une réponse sous forme textuelle ou sous forme de code. Toute réponse doit être écrite dans une ou plusieurs cellules après l’énoncé de chaque exercice.

Vous rendrez un répertoire compressé `tp_bert_nom1_nom2.zip` avec le contenu du répertoire `tp_bert.zip` dont le fichier `tp_bert.ipynb` aura été mis à jour avec vos reponses.

## Données



Nous allons entraîner notre modèle sur une tache de **classification en classes multiples**. Notamment, la tâche est celle de repartir des textes en trois catégories de sentiments:

1. négatif
2. neutre
3. positif

Ces donnés sont collectées dans le jeu de données [FinancialPhraseBank-v1.0
](https://www.researchgate.net/publication/251231364_FinancialPhraseBank-v10), que vous pouvez trouver dans le répertoire de ce TP.

Voici les informations essentielles à la compréhension de la tâche, qui figurent dans le file README.txt :

---

<em>The key arguments for the low utilization of statistical techniques in financial sentiment analysis have been the difficulty of implementation for practical applications and the lack of high quality training data for building such models. Especially in the case of finance and economic texts, annotated collections are a scarce resource and many are reserved for proprietary use only. To resolve the missing training data problem, we present a collection of ∼ 5000 sentences to establish human-annotated standards for benchmarking alternative modeling techniques. 

<em>The objective of the phrase level annotation task was to classify each example sentence into a positive, negative or neutral category by considering only the information explicitly available in the given sentence. Since the study is focused only on financial and economic domains, the annotators were asked to consider the sentences from the view point of an investor only; i.e. whether the news may have positive, negative or neutral influence on the stock price. As a result, sentences which have a sentiment that is not relevant from an economic or financial perspective are considered neutral.

<em>This release of the financial phrase bank covers a collection of 4840 sentences. The selected collection of phrases was annotated by 16 people with adequate background knowledge on financial markets. Three of the annotators were researchers and the remaining 13 annotators were master’s students at Aalto University School of Business with majors primarily in finance, accounting, and economics.

<em>Given the large number of overlapping annotations (5 to 8 annotations per sentence), there are several ways to define a majority vote based gold standard. To provide an objective comparison, we have formed 4 alternative reference datasets based on the strength of majority agreement: 

1. sentences with 100% agreement [file=Sentences_AllAgree.txt]; 
2. sentences with more than 75% agreement [file=Sentences_75Agree.txt]; 
3. sentences with more than 66% agreement [file=Sentences_66Agree.txt]; and 
4. sentences with more than 50% agreement [file=Sentences_50Agree.txt].

<em>All reference datasets are included in the release. The files are in a machine-readable "@"-separated format:

<em>**sentence@sentiment**

<em>where sentiment is either "positive, neutral or negative".

<em>E.g.,  The operating margin came down to 2.4 % from 5.7 % .@negative<em>

---

#### Exercice 1

Nous allons utiliser les phrases contenues dans _Sentences_75Agree.txt_ pour entraîner et tester notre modèle. Pour ce faire, téléchargez le fichier sur la machine Colab (utiliser l’interface à gauche) ou, si vous travaillez sur un _Jupyter Notebook_ en local, placez le fichier _Sentences_75Agree.txt_ dans le même répertoire du _Notebook_.

Ensuite, écrivez la fonction `load_data()` qui lit les phrases contenues dans ce fichier et les sépare de leurs étiquettes. Le résultat de cette fonction, assigné à la variable `data`, doit être une liste de listes, où chaque sous-liste contient une phrase comme premier élément et son étiquette comme deuxième élément. I.e.:

````
data[0][0] == 'A high court in Finland has fined seven local asphalt companies more than   lion ( $ 117 million ) for operating a cartel .'
data[0][1] == 'negative'
````

> Note: l'encodage du fichier _Sentences_75Agree.txt_ est `iso-8859-1`.

In [ ]:
# WRITE FUNCTION load_data HERE

filename = 'Sentences_75Agree.txt'
classes = ['negative', 'neutral', 'positive']
data = load_data(filename, classes)

Testez le résultat de votre fonction en faisant tourner la cellule suivante.

In [ ]:
assert type(data[0][0]) == str, "The first element of every sub-list should be a sentence."
assert data[0][1] in classes, "The second element of each sub-list should belong to one of the three classes available."
assert len(data) == 3453, "The size of data should be of 3453 sentences."
print("Test passed!")

Maintenant, nous allons séparer nos données pour l’entraînement afin de tester notre modèle.

In [ ]:
import random, math
from sklearn import preprocessing

# randomly sample train and test set from data
random.seed(0)
random.shuffle(data)
split = 0.85
num_elem = math.floor(len(data)*split)
train = data[:num_elem]
test = data[num_elem:]
print("Train data size : ", len(train))
print("Test data size : ", len(test))

# build input features and labels
x_train = [t[0] for t in train]
y_train = [t[1] for t in train]
x_test = [t[0] for t in test]
y_test = [t[1] for t in test]

# convert labels into integers ['negative', 'neutral', 'positive'] -> [0, 1, 2]
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

### Baseline


#### Exercice 2

Préalablement à une tache de classification, c’est toujours une bonne idée de vérifier si les données de training et de test sont (plus ou moins) uniformément distribuées par rapport aux classes existantes.

1. Pourquoi ?

2. Quel est le nombre de données d'entraînement et de test correspondant à chaque classe ?

3. Étant donnée la distribution des étiquettes dans le _testset_, quelle sont les attentes par rapport aux performances de notre modèle en terme de _accuracy_ ? 

In [ ]:
# WRITE CODE TO ANSWER THE QUESTIONS HERE,  PRINT ANSWERS OR WRITE THEM IN A TEXT CELL BELOW

#### Exercice 3

Dans la cellule ci-dessous, on entraîne un modèle de [classification naïve bayésienne
 en classes multiples](https://fr.wikipedia.org/wiki/Classification_na%C3%AFve_bay%C3%A9sienne), en sorte d’avoir une performance de référence. Ainsi, l’objectif dans la suite sera de faire mieux de ce naïf.

1. Pourquoi est-qu’on s’attende qu’un modèle de classification type BERT est plus puissante du modèle naïf bayésien ?
2. Dans quelles conditions les modèles non-neuronaux peuvent être plus avantageux ?

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

# transform sentences in bag of words
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(x_train)
x_test_counts = count_vect.transform(x_test)
# transform bag of words in tfidf vectors
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)
x_test_tfidf = tfidf_transformer.fit_transform(x_test_counts)
# train a multinomial Naive-Bayes classifier on the tfidf vectors
classifier = MultinomialNB().fit(x_train_tfidf, y_train)
# use classifier to predict the labels of the test set
predicted = classifier.predict(x_test_tfidf)
# calculate accuracy
print("Accuracy of a simple multinomial Naive-Bayes approach :", sum(predicted == y_test)/len(predicted))

## Tokenisation

La bibliothèque HuggingFace offre des outils pour faire la tokenisation des données textuelles selon les modéles que nous allons utiliser: [see the docs](https://huggingface.co/transformers/tokenizer_summary.html).

Dans ce TP, nous allons utiliser un modèles pré-entraîné qui s’appelle [DistilBERT](https://arxiv.org/pdf/1910.01108.pdf): un encodeur avec une architecture [Transformer](https://arxiv.org/abs/1706.03762) une version distillée de BERT et qui ainsi plus léger en terme de mémoire et plus rapide. Les auteurs le présentent dans leur papier de la manière suivante :

<em>As Transfer Learning from large-scale pre-trained models becomes more prevalent in Natural Language Processing (NLP), operating these large models in on-the-edge and/or under constrained computational training or inference budgets remains challenging. In this work, we propose a method to pre-train a smaller general-purpose language representation model, called DistilBERT, which can then be fine-tuned with good performances on a wide range of tasks like its larger counterparts. While most prior work investigated the use of distillation for building task-specific models, we leverage knowledge distillation during the pre-training phase and show that it is possible to reduce the size of a BERT model by 40%, while retaining 97% of its language understanding capabilities and being 60% faster. To leverage the inductive biases learned by larger models during pre-training, we introduce a triple loss combining language modeling, distillation and cosine-distance losses. Our smaller, faster and lighter model is cheaper to pre-train and we demonstrate its capabilities for on-device computations in a proof-of-concept experiment and a comparative on-device study.

In [ ]:
from transformers import DistilBertTokenizer

MAX_LEN = 512

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', padding=True, truncation=True)

# let's check out how the tokenizer works
for n in range(3):
    # tokenize sentences
    tokenizer_out = tokenizer(x_train[n])
    # convert numerical tokens to alphabetical tokens
    encoded_tok = tokenizer.convert_ids_to_tokens(tokenizer_out.input_ids)
    # decode tokens back to string
    decoded = tokenizer.decode(tokenizer_out.input_ids)
    print(tokenizer_out)
    print(encoded_tok, '\n')
    print(decoded, '\n')
    print('---------------- \n')


#### Exercice 4

1. Qu’est-ce que la sortie du tokeniseur (`tokeniser_out`) représente ? 
2. Quels sont les tokens spéciaux introduits par le tokeniseur? Quel est leur fonction ?
3. Pourquoi certains tokens commencent par ## (par exemple, ##rea ##der) ? Quel intérêt ?
4. Notez que le tokeniseur que l'on utilise a été "entraîné". Pourquoi un tokeniseur comme celui-ci nécessite d’un entraînement préalable à son application ? **Aide**: creuser le sujet Byte Pair Encodings.

#### Exercise 5

BERT (et DistilBERT) gère des séquences de longueur maximale égale à 512 (`MAX_LEN=512`). Vérifier si cette longueur est optimale pour notre tâche. En d'autre termes, vérifiez quelle est la distribution des longueurs des textes que nous devons classifier et s’il serait avantageux de réduire MAX_LEN de façon à réduire le temps de traitement des séquences. 

In [ ]:
# WRITE CODE TO ANSWER THE QUESTION HERE.  PRINT THE ANSWER OR WRITE IT IN A TEXT CELL BELOW

Maintenant que nous avons compris comment le tokeniseur fonctionne, nous pouvons désormais écrire une classe Dataset qui nous servira pour l'entraînement et le test. En effet, le classes Python qui s’occupent de la création de _batches_ et du training nécessitent en entrée une classe de type Dataset, comme la suivante.

**Note** changez MAX_LEN si vous avez trouvé dans l'exercice précédent une valeur qui est plus avantageuse en terme de temps de calcul.

In [ ]:
from torch.utils.data import Dataset, DataLoader

# MAX_LEN =

class MyDataset(Dataset):
    def __init__(self, sentences, labels, tokenizer, max_len):
        # variables that are set when the class is instantiated
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)
  
    def __getitem__(self, item):
        # select the sentence and its class
        sentence = str(self.sentences[item])
        label = self.labels[item]
        # tokenize the sencence
        tokenizer_out = self.tokenizer(
            sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
            )
        # return a dictionary with the output of the tokenizer and the label
        return  {
            'input_ids': tokenizer_out['input_ids'].flatten(),
            'attention_mask': tokenizer_out['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


# instantiate two MyDataset objects
train_dataset = MyDataset(x_train, y_train, tokenizer, MAX_LEN)
test_dataset = MyDataset(x_test, y_test, tokenizer, MAX_LEN)

## Modèle

C’est le moment de comprendre comment DistilBERT fonctionne. Hugginface nous offre plusieurs modèles pré-entraînes de type DistilBERT, nous allons utiliser [distilbert-base-uncased](https://huggingface.co/distilbert-base-uncased), qui a une architecture profonde de 66’362’880 paramétres et qui a été pré-entraîné sur le jeu de données BookCorpus pour 90 heures avec huit GPUs de 16 GB de mémoire. Cela nous permettra d’obtenir de très bons résultat en attachant un classificateur très léger (un seul layer) au top de DistilBERT, qui fera le plus gros du travail, c’est-à-dire l’encodage de nos textes. Notre classificateur nécessitera d’être entraîné pendant quelques minutes seulement. 

On peut trouver les noms de tous les autres modèles pré-entraînés offerts par HuggingFace [à cette addresse](https://huggingface.co/models). 

Pour télécharger un modèle pré-entraîné on utilise l'option `.from_pretrained`:

In [ ]:
from transformers import DistilBertModel

PRE_TRAINED_MODEL_NAME = 'distilbert-base-uncased'

distilbert = DistilBertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)


Nous pouvons encoder les séquences de textes avec une passe en avant de distilBERT, qui nous renvoie les représentations cachés de la dernière couche.

**Note** : la dimension des couches de `distilbert-base-uncased` est de 768 neurones 

**Note2** : nous utilisons la fonction `.unsqueeze(0)` parce que normalement on passe au modèle des batches, alors que cette fois ci ce n’est qu'un seul élement de la classe *MyDataset*. Avec `.unsqueeze(0)`, nous le traitons comme un batch de taille 1.

In [ ]:
first_sent = train_dataset[0]

hidden_state = distilbert(
    input_ids=first_sent['input_ids'].unsqueeze(0), attention_mask=first_sent['attention_mask'].unsqueeze(0)
    )

hidden_state[0].shape

In [ ]:
distilbert.config

#### Exercice 6

Nous avons maintenant les éléments nécessaires à la construction d'un modèle de classification en classes multiples basé sur DistilBERT. 

Nous allons définire une classe `DistilBertForSentimentClassification` pour notre modèle. Cette classe importe la classe DistilBertPreTrainedModel, qui a son tour importe la classe Pytorch nn.Module. Comme vous pouvez le voir dans la documentation Pytorch, la classe Module définie une méthode `forward()`, qu'on essaye de remplacer dans la définition de `DistilBertForSentimentClassification`. La méthode `forward()` définie le comportement du modèle lorsqu'on lui donne des données en entrée. Par exemple, vous pouvez considerer la commande `encoder_output = self.encoder(...input...)` (voir code ci-dessous) comme équivalente à `encoder_output = self.encoder.forward(...input...)` , parce que `self.encoder` est une instance de la classe `DistilBertModel`, qui est aussi basée sur `nn.Module` (voir documentation/code HuggingFace pour plus de détails).
    

Concernant le code ci-dessous :

1. Quelle est la fonction de cet extrait de code ? Quel est l'intérêt?
	```Python
	if freeze_encoder:
		for param in self.encoder.parameters():
			param.requires_grad = False
	```

2. Pourquoi on ne garde que la représentation cachée (vecteur encodé) du premier token de chaque séquence avec la commande `pooled_output = hidden_state[:, 0, :]` ?

    - **aide**: Pour comprendre pourquoi on peut ne garder que l'encodage du premier token de chaque phrase, il faut essayer de comprendre quelle information est rapresentée par ce vecteur. Pour ce faire, il est important de comprendre la méthode d'entraînement des modèles type BERT, où le token `[CLS]` à un role très spécifique. Vous trouverez les informations nécessaires à comprendre cela dans le paper [BERT](https://arxiv.org/pdf/1810.04805.pdf). Si vous trouvez le papier difficile, vous pouvez aussi trouver des explications simplifiées et résumées sur youtube/medium/etc.

3. Compléter l’extrait de code sous-jacente à `if labels is not None:` de façon à calculer la fonction de perte (*loss  function*) du modèle lorsque les cibles sont passées à la fonction `forward`

    - **aide**: Vous n'avez qu'à écrire une ligne de code: `loss = torch.nn.functional.???(???,???)` en choisissant la bonne fonction de perte. Les fonctions de perte implémentées en Pytorch sont répertoriées ici: https://pytorch.org/docs/stable/nn.html#loss-functions. Vous trouvez en ligne les informations pour choisir la fonction de perte la plus adaptée à la tache. Par exemple, en tapant "pytorch loss functions" sur Google, on trouve ces deux articles: https://neptune.ai/blog/pytorch-loss-functions et https://medium.com/udacity-pytorch-challengers/a-brief-overview-of-loss-functions-in-pytorch-c0ddb78068f7 .



In [ ]:
from transformers import DistilBertPreTrainedModel, DistilBertConfig


PRE_TRAINED_MODEL_NAME = 'distilbert-base-uncased'
FREEZE_PRETRAINED_MODEL = True

class DistilBertForSentimentClassification(DistilBertPreTrainedModel):
    def __init__(self, config, num_labels, freeze_encoder=False):
        # instantiate the parent class DistilBertPreTrainedModel
        super().__init__(config)
        # instantiate num. of classes
        self.num_labels = num_labels
        # instantiate and load a pretrained DistilBERT model as encoder
        self.encoder = DistilBertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        # [Q1] freeze the encoder parameters if required 
        if freeze_encoder:
          for param in self.encoder.parameters():
              param.requires_grad = False
        # the classifier: a feed-forward layer attached to the encoder's head
        self.classifier = torch.nn.Linear(
            in_features=config.dim, out_features=self.num_labels, bias=True)
        # instantiate a dropout function for the classifier's input
        self.dropout = torch.nn.Dropout(p=0.1)


    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
    ):
        # encode a batch of sequences with DistilBERT
        encoder_output = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )
        # extract the hidden representations from the encoder output
        hidden_state = encoder_output[0]  # (bs, seq_len, dim)
        # only select the encoding corresponding to the first token
        # of each sequence in the batch [Q2]
        pooled_output = hidden_state[:, 0, :]  # (bs, dim)
        # apply dropout
        pooled_output = self.dropout(pooled_output)  # (bs, dim)
        # feed into the classifier
        logits = self.classifier(pooled_output)  # (bs, dim)

        outputs = (logits,) + encoder_output[1:]
        
        if labels is not None:
          # [Q3] COMPLETE CODE HERE
          # loss = ...

          # aggregate outputs
          outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)


# instantiate model
model = DistilBertForSentimentClassification(
    config=distilbert.config, num_labels=len(classes),
    freeze_encoder = FREEZE_PRETRAINED_MODEL
    )

# print info about model's parameters
total_params = sum(p.numel() for p in model.parameters())
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
trainable_params = sum([np.prod(p.size()) for p in model_parameters])
print('model total params: ', total_params)
print('model trainable params: ', trainable_params)
print('\n', model)


## Entraînement

*Let's train our classifier!*

Le moment est arrivé. Heureusement *HuggingFace* nous fournit la classe Python nécessaire à gérer le training : `Trainer`. Nous lui passons les hyper-paramètres avec la classe `TrainingArgument`. Nous allons aussi lui passer les métrique que nous désirons pour l’évaluation du modèle en phase de test :

1. justesse (accuracy)
2. précision (precision)
3. rappel (recall)
4. f1


In [ ]:
# clean logs and results directory from old files
# this could be useful if we were running the cells below multiple times
!rm -r ./logs ./results

In [ ]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',          
    logging_dir='./logs',
    logging_first_step=True,
    logging_steps=50,
    num_train_epochs=16,              
    per_device_train_batch_size=8,  
    learning_rate=5e-5,
    weight_decay=0.01        
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    compute_metrics=compute_metrics
)

Pour visualiser l’évolution de la *loss* tout au long de l'entraînement, nous pouvons utiliser *TensorBoard *:

In [ ]:
%tensorboard --logdir logs

In [ ]:
train_results = trainer.train()

In [ ]:
test_results = trainer.predict(test_dataset=test_dataset)

In [ ]:
print('Predictions: \n', test_results.predictions)
print('\nAccuracy: ', test_results.metrics['eval_accuracy'])
print('Precision: ', test_results.metrics['eval_precision'])
print('Recall: ', test_results.metrics['eval_recall'])
print('F1: ', test_results.metrics['eval_f1'])
print(classes)

Si nous avons fait les choses bien, la performance (*accuracy*) de notre modèle dans la classification des phraces qu’il n’a jamais vu est d'environ 70.0%. Cela n'est pas très satisfaisant si on considère :

1. que la distribution des classes dans notre ensemble de test n’était pas équilibrée (c.f. Exercice 2)
2. qu'un modèle de classification naïve bayésienne obtient une accuracy similaire, voir supérieure (c.f. Exercice 3)
 

### Améliorer les performances

#### Exercice 7

1. Comment est-ce qu'on peut améliorer le résultat du modèle ?
2. Ré-entraînez le modèle avec la nouvelle stratégie et calculez ses performances avec les métriques utilisées avant. Quelle _accuracy_ vous arrivez à obtenir ?

**Aide** : quels hyperparamétres influencent l'entrainement du modèle ? Rappelez vous aussi que nous avons congelé l'encodeur.

In [ ]:
# WRITE CODE TO ANSWER THE QUESTION HERE.  PRINT THE ANSWER OR WRITE IT IN A TEXT CELL BELOW

Si nous somme satisfaits de la performance de notre modèle, nous pouvons le sauvegarder pour l’utiliser dans le prochain exercice : 

In [ ]:
MODEL_PATH = './my_model'
trainer_best.save_model(MODEL_PATH)

## Prédictions

#### Exercice 8

En utilisant le modèle que l’on vient de sauvegarder, prédire la classe à laquelle appartient les phrases suivantes :

````
  "CocaCola saw its share price dropping of more than 25% this semester.",
  "Despite most of the company's sales are taking place in China, the shareholders decided not to relocate the production there." ,
  "This year's profits quintuplicated with respect to the last year."
````

1. Quelles classes a prédit le modèle ?
2. Avec quelle probabilité ?

In [ ]:
# WRITE CODE TO ANSWER THE QUESTION HERE.  PRINT THE ANSWER OR WRITE IT IN A TEXT CELL BELOW

## Changer de modèle

#### Exercice 9 (bonus)

Expérimentez avec d’autres modèles pré-entraînés. La liste des modèles pré-entraînés est disponible ici: https://huggingface.co/models.

- Qu’est-ce que vous avez essayé ? Laissez votre code dans des cellules ci-dessous. 
- Qu’est-ce que vous avez appris ?